In [1]:
import requests
import json
import pandas as pd
from functools import reduce
from pandas.io.json import json_normalize

URL = 'http://real_search:8983/solr/dbnews/select'
T_URL = 'http://search:8983/solr/news/select'
def get_search_result(search_words, from_dt, to_dt):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[' + from_dt + 'T00:00:00Z TO ' + to_dt + 'T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result


def get_tot_search_result(search_words, from_dt, to_dt):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[' + from_dt + 'T00:00:00Z TO ' + to_dt + 'T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(T_URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result

In [2]:
week1 = ['2017-03-27','2017-04-02']
week2 = ['2017-04-03','2017-04-09']
week3 = ['2017-04-10','2017-04-16']
week4 = ['2017-04-17','2017-04-23']
week5 = ['2017-04-24','2017-04-30']

week = [
    ['2017-03-27','2017-04-02'], 
    ['2017-04-03','2017-04-09'],
    ['2017-04-10','2017-04-16'],
    ['2017-04-17','2017-04-23'],
    ['2017-04-24','2017-04-30']   
       ]

from_dt = '2017-05-09'
to_dt = '2017-05-10'

In [3]:
df_tot_moon_result = json_normalize(get_tot_search_result('문재인', from_dt, to_dt))
df_tot_ahn_result = json_normalize(get_tot_search_result('안철수',  from_dt, to_dt))
df_tot_hong_result = json_normalize(get_tot_search_result('홍준표', from_dt, to_dt))
df_tot_you_result = json_normalize(get_tot_search_result('유승민',  from_dt, to_dt))
df_tot_sim_result = json_normalize(get_tot_search_result('심상정',  from_dt, to_dt))
    
df_moon_result = json_normalize(get_search_result('문재인', from_dt, to_dt))
df_ahn_result = json_normalize(get_search_result('안철수',  from_dt, to_dt))
df_hong_result = json_normalize(get_search_result('홍준표', from_dt, to_dt))
df_you_result = json_normalize(get_search_result('유승민',  from_dt, to_dt))
df_sim_result = json_normalize(get_search_result('심상정',  from_dt, to_dt))

In [4]:
df_moon_result['tot'] = df_moon_result['n_shares'].fillna(0) + df_moon_result['o_shares'].fillna(0) 
df_ahn_result['tot'] = df_ahn_result['n_shares'].fillna(0) + df_ahn_result['o_shares'].fillna(0) 
df_hong_result['tot'] = df_hong_result['n_shares'].fillna(0) + df_hong_result['o_shares'].fillna(0) 
df_you_result['tot'] = df_you_result['n_shares'].fillna(0) + df_you_result['o_shares'].fillna(0) 
df_sim_result['tot'] = df_sim_result['n_shares'].fillna(0) + df_sim_result['o_shares'].fillna(0) 

In [5]:
df_moon_result['can_no'] = '01'
df_ahn_result['can_no'] = '02'
df_hong_result['can_no'] = '03'
df_you_result['can_no'] = '04'
df_sim_result['can_no'] = '05'

df_all_result=df_moon_result
df_all_result=df_all_result.append([df_ahn_result, df_hong_result, df_you_result, df_sim_result], ignore_index=True)
df_all_result['can_no'].unique()
#df_all_result.tail()

array(['01', '02', '03', '04', '05'], dtype=object)

In [6]:
import io
import itertools
import networkx as nx
import os
from konlpy.tag import Mecab
from konlpy.tag import Twitter
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from gensim.summarization.commons import build_graph as _build_graph
from gensim.summarization.commons import remove_unreachable_nodes as _remove_unreachable_nodes
from gensim.summarization.pagerank_weighted import pagerank_weighted as _pagerank

#def filter_for_tags(tagged, tags=['NNP', 'NNG']):
def filter_for_tags(tagged, tags=['Noun', 'Verb']):
    """Apply syntactic filters based on POS tags."""
    return [item for item in tagged if item[1] in tags]

def normalize(tagged):
    """Return a list of tuples with the first item's periods removed."""
    return [(item[0].replace('.', ''), item[1]) for item in tagged]

def build_graph(nodes):
    """Return a networkx graph instance.
    :param nodes: List of hashables that represent the nodes of a graph.
    """
    #gr = nx.Graph()  # initialize an undirected graph
    #gr.add_nodes_from(nodes)
    graph = _build_graph(nodes)
    nodePairs = list(itertools.combinations(nodes, 2))

    # add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        firstString = pair[0]
        secondString = pair[1]
        #levDistance = levenshtein_distance(firstString, secondString)
        #gr.add_edge(firstString, secondString, weight=levDistance)
        edge = (firstString, secondString)
        
        if graph.has_node(firstString) and graph.has_node(secondString) and not graph.has_edge(edge):
            graph.add_edge(edge)
        #graph.add_edge(edge)

    return graph

def levenshtein_distance(first, second):
    """Return the Levenshtein distance between two strings.
    Based on:
        http://rosettacode.org/wiki/Levenshtein_distance#Python
    """
    if len(first) > len(second):
        first, second = second, first
    distances = range(len(first) + 1)
    for index2, char2 in enumerate(second):
        new_distances = [index2 + 1]
        for index1, char1 in enumerate(first):
            if char1 == char2:
                new_distances.append(distances[index1])
            else:
                new_distances.append(1 + min((distances[index1],
                                             distances[index1 + 1],
                                             new_distances[-1])))
        distances = new_distances
    return distances[-1]

def extract_key_phrases(text):
    """Return a set of key phrases.
    :param text: A string.
    """
    #t = Kkma()
    #t = Komoran()
    #t=Mecab()
    t = Twitter()
    tags_ko = t.pos(text)    
    
    textlist = [x[0] for x in tags_ko]
    tags_ko = filter_for_tags(tags_ko)
    
    tags_ko = normalize(tags_ko)
    word_set_list=list(set([x[0] for x in tags_ko]))
    #word_set_list = list(tags_ko)
    
    graph = build_graph(word_set_list)
    _remove_unreachable_nodes(graph)
    
    pagerank_scores = _pagerank(graph)
    
    '''
    calculated_page_rank = nx.pagerank(graph, weight='weight')

    # most important words in ascending order of importance
    keyphrases = sorted(calculated_page_rank, key=calculated_page_rank.get,reverse=True)
    
    one_third = len(word_set_list) // 3
    keyphrases = keyphrases[0:one_third + 1]

    
    # take keyphrases with multiple words into consideration as done in the
    # paper - if two words are adjacent in the text and are selected as
    # keywords, join them together
    modified_key_phrases = set([])
    # keeps track of individual keywords that have been joined to form a
    # keyphrase
    dealt_with = set([])
    i = 0
    j = 1
    while j < len(textlist):
        first = textlist[i]
        second = textlist[j]
        if first in keyphrases and second in keyphrases:
            keyphrase = first + ' ' + second
            modified_key_phrases.add(keyphrase)
            dealt_with.add(first)
            dealt_with.add(second)
        else:
            if first in keyphrases and first not in dealt_with:
                modified_key_phrases.add(first)

            # if this is the last word in the text, and it is a keyword, it
            # definitely has no chance of being a keyphrase at this point
            if j == len(textlist) - 1 and second in keyphrases and \
                    second not in dealt_with:
                modified_key_phrases.add(second)

        i = i + 1
        j = j + 1
        
    return modified_key_phrases
    '''

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-6-c56eeac69b1f>, line 127)

In [51]:
def extract_sentence(text):
    text = text.replace('\n \n', '\n\n').replace('\t', '')
    sentences = text.split('\n\n')
    return sentences[:-4]

In [52]:
df_ten = df_tot_moon_result[df_tot_moon_result['oid']=='001']
#df_ten['article'] = df_ten['article'].str.replace(r'\t|\n|\/\/[a-zA-Z가-힣 \n]+|{}|_flash_removeCallback\(\)|[a-zA-Z0-9]+@yonhapnews.co.kr|\[[a-zA-Z가-힣]\]','')
df_ten['article'] = df_ten['article'].str.replace(r'\([가-힣=]+\)|\[[a-z0-9가-힣]+\]|\[[a-zA-Z0-9가-힣\(\)\= ]+\]', '')
df_ten['article'] = df_ten.apply(lambda row: extract_sentence(row['article']), axis=1)


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [196]:
df_ten['twitter_keyphrase'] = df_ten.apply(lambda row: extract_key_phrases(row['article']), axis=1)

http://stackoverflow.com/questions/24203106/pandas-group-by-calendar-week-then-plot-grouped-barplots-for-the-real-datetime

In [197]:
df_ten[['title', 'twitter_keyphrase']].to_csv('twitter_keywords.tsv',sep='\t')

In [21]:
from konlpy.tag import Mecab; t = Mecab()
documents = df_ten['article']
pos = lambda d: ['/'.join(p) for p in t.pos(d) if p[1] in ['NNP','NNG']]
texts_ko = [pos(doc) for doc in documents]
print(texts_ko[0])

NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)


In [44]:
from gensim import corpora
from gensim import models
dictionary = corpora.Dictionary(texts_ko) 
tf = [dictionary.doc2bow(doc) for doc in texts_ko] 
tfidf_model = models.TfidfModel(tf)
tfidf = tfidf_model[tf]


ntopics, nwords = 10, 10
lsi = models.lsimodel.LsiModel(tfidf, id2word=dictionary, num_topics=ntopics)
for idx, i in enumerate(lsi.print_topics(num_topics=ntopics, num_words=nwords)):
    print(idx, ':', i)

0 : (0, '0.310*"포인트/NNG" + 0.254*"대표/NNG" + 0.200*"조사/NNG" + 0.173*"지사/NNG" + 0.165*"경선/NNG" + 0.139*"호남/NNG" + 0.132*"여론/NNG" + 0.122*"안철수/NNP" + 0.113*"충청/NNP" + 0.106*"시장/NNG"')
1 : (1, '0.397*"포인트/NNG" + 0.272*"조사/NNG" + -0.174*"경선/NNG" + 0.172*"여론/NNG" + 0.159*"한국갤럽/NNP" + -0.123*"호남/NNG" + 0.115*"하락/NNG" + -0.111*"영남/NNP" + 0.108*"홈페이지/NNG" + 0.103*"상승/NNG"')
2 : (2, '-0.184*"경선/NNG" + -0.160*"충청/NNP" + -0.149*"과반/NNG" + -0.141*"득표/NNG" + -0.135*"순회/NNG" + -0.133*"호남/NNG" + -0.130*"지사/NNG" + -0.124*"누적/NNG" + -0.116*"영남/NNP" + -0.112*"시장/NNG"')
3 : (3, '0.433*"경찰/NNG" + 0.424*"현수막/NNG" + 0.261*"구청장/NNG" + 0.192*"훼손/NNG" + 0.179*"발견/NNG" + 0.133*"수사/NNG" + 0.121*"신연희/NNP" + 0.119*"비방/NNG" + 0.111*"고발/NNG" + 0.107*"혐의/NNG"')
4 : (4, '-0.227*"여성/NNG" + -0.182*"경남/NNP" + 0.163*"북한/NNP" + -0.136*"선언/NNG" + 0.134*"대표/NNG" + -0.121*"펀드/NNG" + 0.118*"박/NNP" + -0.115*"충북/NNP" + -0.109*"체육/NNG" + -0.108*"평등/NNG"')
5 : (5, '-0.420*"동성애/NNG" + 0.215*"펀드/NNG" + -0.204*"차별/NNG" + -0.194*"반대/NN

In [45]:
import numpy as np; np.random.seed(42)  # optional
lda_ko = models.ldamodel.LdaModel(tfidf, id2word=dictionary, num_topics=ntopics)
for idx, i in enumerate(lda_ko.print_topics(num_topics=ntopics, num_words=nwords)):
    print(idx, ':', i)

0 : (0, '0.002*"영남/NNP" + 0.001*"충북/NNP" + 0.001*"단체/NNG" + 0.001*"토론/NNG" + 0.001*"유세/NNG" + 0.001*"준비/NNG" + 0.001*"성장/NNG" + 0.001*"포인트/NNG" + 0.001*"롯데/NNP" + 0.001*"대북/NNG"')
1 : (1, '0.002*"대표/NNG" + 0.002*"경선/NNG" + 0.002*"포인트/NNG" + 0.002*"조사/NNG" + 0.002*"호남/NNG" + 0.001*"지사/NNG" + 0.001*"시장/NNG" + 0.001*"여론/NNG" + 0.001*"한국갤럽/NNP" + 0.001*"발췌/NNG"')
2 : (2, '0.002*"호남/NNG" + 0.002*"대표/NNG" + 0.002*"경선/NNG" + 0.002*"승리/NNG" + 0.002*"광주/NNP" + 0.001*"지지/NNG" + 0.001*"선출/NNG" + 0.001*"박/NNP" + 0.001*"교체/NNG" + 0.001*"압도/NNG"')
3 : (3, '0.002*"사교육/NNG" + 0.002*"의원/NNG" + 0.001*"대표/NNG" + 0.001*"개헌/NNG" + 0.001*"지사/NNG" + 0.001*"박/NNP" + 0.001*"기호/NNG" + 0.001*"포인트/NNG" + 0.001*"천안/NNP" + 0.001*"김/NNP"')
4 : (4, '0.002*"포인트/NNG" + 0.002*"조사/NNG" + 0.002*"지지/NNG" + 0.002*"대표/NNG" + 0.001*"여론/NNG" + 0.001*"안/NNG" + 0.001*"안철수/NNP" + 0.001*"당/NNG" + 0.001*"대결/NNG" + 0.001*"북한/NNP"')
5 : (5, '0.001*"여성/NNG" + 0.001*"영화/NNG" + 0.001*"의원/NNG" + 0.001*"동성애/NNG" + 0.001*"문서/NNG" + 0.001*"

In [51]:
bow = tfidf_model[dictionary.doc2bow(texts_ko[0])]
sorted(lsi[bow], key=lambda x: x[1], reverse=True)

[(2, 0.088369030764647388),
 (0, 0.072795923341513108),
 (8, 0.021276687512595389),
 (5, 0.017721111843685456),
 (4, 0.015683585098546754),
 (9, 0.0083102832378413368),
 (6, -0.0027185925612517592),
 (7, -0.013347665465319878),
 (3, -0.014933605037341039),
 (1, -0.06658082927786671)]

In [52]:
sorted(lda_ko[bow], key=lambda x: x[1], reverse=True)

[(6, 0.90021249581476592),
 (2, 0.01109253973458715),
 (9, 0.011088993001316034),
 (1, 0.011088827038805866),
 (0, 0.011088328941173221),
 (4, 0.011088284813027458),
 (8, 0.011086375491255095),
 (3, 0.011085330182107261),
 (7, 0.011084593412921161),
 (5, 0.01108423157004068)]

In [65]:
df_all = df_tot_ahn_result
#df_ten['article'] = df_ten['article'].str.replace(r'\t|\n|\/\/[a-zA-Z가-힣 \n]+|{}|_flash_removeCallback\(\)|[a-zA-Z0-9]+@yonhapnews.co.kr|\[[a-zA-Z가-힣]\]','')
df_all['article'] = df_all['article'].str.replace(r'\([가-힣=]+\)|\[[a-z0-9가-힣]+\]|\[[a-zA-Z0-9가-힣\(\)\= ]+\]', '')
df_all['article'] = df_all.apply(lambda row: extract_sentence(row['article']), axis=1)


In [66]:
documents = df_all['article']
#pos = lambda d: ['/'.join(p) for p in t.pos(d)]
#texts_ko = [pos(doc) for doc in documents]
#print(texts_ko[0])
corpus_ko = []
for doc in documents:
    corpus_ko.extend(doc)

In [67]:
def noun_tagger(text):
    #return [pos[0] for pos in t.pos(text) if pos[1] in ['SL','SH','SN','NN','NNG', 'NNP']]
    return [pos[0] for pos in t.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    #if pos[1] in ['NNP', 'NNG']:

http://www.ultravioletanalytics.com/2016/11/18/tf-idf-basics-with-pandas-scikit-learn/

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [68]:
import numpy as np;
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger, ngram_range=(2,5), min_df=.0025, max_df=.1)
vect_weights = vect.fit_transform(corpus_ko)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
weights = np.asarray(vect_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vect.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(30)

,term,weight
244,경향신문 sns,0.020897
860,안 후보,0.016384
866,안철수 국민,0.009859
474,대선 후보,0.009326
605,문 후보,0.009161
883,안철수 후보,0.008864
38,19 대,0.008190
1234,홍 후보,0.007978
867,안철수 국민 당,0.007685
891,여론 조사,0.007463


In [69]:
weights_df.sort_values(by='weight', ascending=False).head(60)

,term,weight
244,경향신문 sns,0.020897
860,안 후보,0.016384
866,안철수 국민,0.009859
474,대선 후보,0.009326
605,문 후보,0.009161
883,안철수 후보,0.008864
38,19 대,0.008190
1234,홍 후보,0.007978
867,안철수 국민 당,0.007685
891,여론 조사,0.007463


In [214]:
t.pos("4차 산업혁명")

[('4', 'SN'), ('차', 'NNBC'), ('산업', 'NNG'), ('혁명', 'NNG')]

In [169]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cvec = CountVectorizer(tokenizer=noun_tagger, min_df=.0025, max_df=.1, ngram_range=(2,5))
cvec.fit(corpus_ko)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.1, max_features=None, min_df=0.0025,
        ngram_range=(2, 5), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function noun_tagger at 0x7f3088eb6268>,
        vocabulary=None)

In [89]:
title_pos = lambda d: ['/'.join(p) for p in t.pos(d) if p[1] in ['NNP','NNG']]
moon_titles=df_moon_result['title'].str.replace(r'\([가-힣=]+\)|\[[a-zA-Z0-9가-힣-]+\]|\[[a-zA-Z0-9가-힣\(\)\= ]+\]', '')
df_moon_result['pos_title'] = [title_pos(title) for title in moon_titles]

In [91]:
df_moon_result[['title','pos_title']]

,title,pos_title
0,"문재인, 오늘부터 본격적인 사전 투표 캠페인 진행","[문재인/NNP, 오늘/NNG, 본격/NNG, 사전/NNG, 투표/NNG, 캠페인/..."
1,"문재인 재벌개혁 3대 키워드, ‘법집행·시장압력·집중억제’","[문재인/NNP, 재벌/NNG, 개혁/NNG, 키워드/NNP, 법/NNG, 집행/N..."
2,[사진]문재인 신촌 유세 '지지의 인파',"[문재인/NNP, 신촌/NNP, 유세/NNG, 지지/NNG, 인파/NNG]"
3,"박지원 ""막말 문재인·홍준표는 대통령 돼선 안 돼""","[박지원/NNP, 막말/NNG, 문재인/NNP, 홍준표/NNP, 대통령/NNG]"
4,"[2017 시민의 선택]문재인, 승기 잡고 ‘투대문’으로","[문재인/NNP, 승기/NNG, 투대/NNG, 문/NNG]"
5,문재인의 밝힌 5대 인생 키워드... ‘피난민 아들`부터 `소명`까지,"[문재인/NNP, 인생/NNG, 키워드/NNP, 피난민/NNG, 아들/NNG, 소명..."
6,"[대선후보 토론회] 문재인 vs 홍준표, 강성귀족노조 논쟁","[문재인/NNP, 홍준표/NNP, 강성/NNG, 귀족/NNG, 노조/NNG, 논쟁/..."
7,"[대선후보 토론회] 문재인 vs 안철수, 국무총리 신속 지명? 국회 추천?","[문재인/NNP, 안철수/NNP, 국무총리/NNG, 신속/NNG, 지명/NNG, 국..."
8,"[대선후보 토론회] 문재인 vs 심상정, 연간 70조 원 증세라고?","[문재인/NNP, 심상정/NNP, 연간/NNG, 증세/NNG]"
9,"[대선후보 토론회] 문재인 vs 안철수, 사드 비용*한미 FTA 논쟁","[문재인/NNP, 안철수/NNP, 사드/NNP, 비용/NNG, 한/NNP, 미/NN..."


http://scikit-learn.org/stable/auto_examples/text/document_clustering.html#sphx-glr-auto-examples-text-document-clustering-py

https://www.youtube.com/watch?v=znfy3T9OiAQ

In [171]:
import re
#_stopwords = ["아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "으로", "로", "에게", "뿐이다", "의거하여", "근거하여", "입각하여", "기준으로", "예하면", "예를", "들면", "들자면", "저", "소인", "소생", "저희", "지말고", "하지마", "하지마라", "다른", "물론", "또한", "그리고", "비길수", "없다", "해서는", "안된다", "뿐만", "아니라", "만이", "아니다", "만은", "아니다", "막론하고", "관계없이", "그치지", "않다", "그러나", "그런데", "하지만", "든간에", "논하지", "않다", "따지지", "않다", "설사", "비록", "더라도", "아니면", "만", "못하다", "하는", "편이", "낫다", "불문하고", "향하여", "향해서", "향하다", "쪽으로", "틈타", "이용하여", "타다", "오르다", "제외하고", "이", "외에", "이", "밖에", "하여야", "비로소", "한다면", "몰라도", "외에도", "이곳", "여기", "부터", "기점으로", "따라서", "할", "생각이다", "하려고하다", "이리하여", "그리하여", "그렇게", "함으로써", "하지만", "일때", "할때", "앞에서", "중에서", "보는데서", "으로써", "로써", "까지", "해야한다", "일것이다", "반드시", "할줄알다", "할수있다", "할수있어", "임에", "틀림없다", "한다면", "등", "등등", "제", "겨우", "단지", "다만", "할뿐", "딩동", "댕그", "대해서", "대하여", "대하면", "훨씬", "얼마나", "얼마만큼", "얼마큼", "남짓", "여", "얼마간", "약간", "다소", "좀", "조금", "다수", "몇", "얼마", "지만", "하물며", "또한", "그러나", "그렇지만", "하지만", "이외에도", "대해", "말하자면", "뿐이다", "다음에", "반대로", "반대로", "말하자면", "이와", "반대로", "바꾸어서", "말하면", "바꾸어서", "한다면", "만약", "그렇지않으면", "까악", "툭", "딱", "삐걱거리다", "보드득", "비걱거리다", "꽈당", "응당", "해야한다", "에", "가서", "각", "각각", "여러분", "각종", "각자", "제각기", "하도록하다", "와", "과", "그러므로", "그래서", "고로", "한", "까닭에", "하기", "때문에", "거니와", "이지만", "대하여", "관하여", "관한", "과연", "실로", "아니나다를가", "생각한대로", "진짜로", "한적이있다", "하곤하였다", "하", "하하", "허허", "아하", "거바", "와", "오", "왜", "어째서", "무엇때문에", "어찌", "하겠는가", "무슨", "어디", "어느곳", "더군다나", "하물며", "더욱이는", "어느때", "언제", "야", "이봐", "어이", "여보시오", "흐흐", "흥", "휴", "헉헉", "헐떡헐떡", "영차", "여차", "어기여차", "끙끙", "아야", "앗", "아야", "콸콸", "졸졸", "좍좍", "뚝뚝", "주룩주룩", "솨", "우르르", "그래도", "또", "그리고", "바꾸어말하면", "바꾸어말하자면", "혹은", "혹시", "답다", "및", "그에", "따르는", "때가", "되어", "즉", "지든지", "설령", "가령", "하더라도", "할지라도", "일지라도", "지든지", "몇", "거의", "하마터면", "인젠", "이젠", "된바에야", "된이상", "만큼 어찌됏든", "그위에", "게다가", "점에서", "보아", "비추어", "보아", "고려하면", "하게될것이다", "일것이다", "비교적", "좀", "보다더", "비하면", "시키다", "하게하다", "할만하다", "의해서", "연이서", "이어서", "잇따라", "뒤따라", "뒤이어", "결국", "의지하여", "기대여", "통하여", "자마자", "더욱더", "불구하고", "얼마든지", "마음대로", "주저하지", "않고", "곧", "즉시", "바로", "당장", "하자마자", "밖에", "안된다", "하면된다", "그래", "그렇지", "요컨대", "다시", "말하자면", "바꿔", "말하면", "즉", "구체적으로", "말하자면", "시작하여", "시초에", "이상", "허", "헉", "허걱", "바와같이", "해도좋다", "해도된다", "게다가", "더구나", "하물며", "와르르", "팍", "퍽", "펄렁", "동안", "이래", "하고있었다", "이었다", "에서", "로부터", "까지", "예하면", "했어요", "해요", "함께", "같이", "더불어", "마저", "마저도", "양자", "모두", "습니다", "가까스로", "하려고하다", "즈음하여", "다른", "다른", "방면으로", "해봐요", "습니까", "했어요", "말할것도", "없고", "무릎쓰고", "개의치않고", "하는것만", "못하다", "하는것이", "낫다", "매", "매번", "들", "모", "어느것", "어느", "로써", "갖고말하자면", "어디", "어느쪽", "어느것", "어느해", "어느", "년도", "라", "해도", "언젠가", "어떤것", "어느것", "저기", "저쪽", "저것", "그때", "그럼", "그러면", "요만한걸", "그래", "그때", "저것만큼", "그저", "이르기까지", "할", "줄", "안다", "할", "힘이", "있다", "너", "너희", "당신", "어찌", "설마", "차라리", "할지언정", "할지라도", "할망정", "할지언정", "구토하다", "게우다", "토하다", "메쓰겁다", "옆사람", "퉤", "쳇", "의거하여", "근거하여", "의해", "따라", "힘입어", "그", "다음", "버금", "두번째로", "기타", "첫번째로", "나머지는", "그중에서", "견지에서", "형식으로", "쓰여", "입장에서", "위해서", "단지", "의해되다", "하도록시키다", "뿐만아니라", "반대로", "전후", "전자", "앞의것", "잠시", "잠깐", "하면서", "그렇지만", "다음에", "그러한즉", "그런즉", "남들", "아무거나", "어찌하든지", "같다", "비슷하다", "예컨대", "이럴정도로", "어떻게", "만약", "만일", "위에서", "서술한바와같이", "인", "듯하다", "하지", "않는다면", "만약에", "무엇", "무슨", "어느", "어떤", "아래윗", "조차", "한데", "그럼에도", "불구하고", "여전히", "심지어", "까지도", "조차도", "하지", "않도록", "않기", "위하여", "때", "시각", "무렵", "시간", "동안", "어때", "어떠한", "하여금", "네", "예", "우선", "누구", "누가", "알겠는가", "아무도", "줄은모른다", "줄은", "몰랏다", "하는", "김에", "겸사겸사", "하는바", "그런", "까닭에", "한", "이유는", "그러니", "그러니까", "때문에", "그", "너희", "그들", "너희들", "타인", "것", "것들", "너", "위하여", "공동으로", "동시에", "하기", "위하여", "어찌하여", "무엇때문에", "붕붕", "윙윙", "나", "우리", "엉엉", "휘익", "윙윙", "오호", "아하", "어쨋든", "만", "못하다    하기보다는", "차라리", "하는", "편이", "낫다", "흐흐", "놀라다", "상대적으로", "말하자면", "마치", "아니라면", "쉿", "그렇지", "않으면", "그렇지", "않다면", "안", "그러면", "아니었다면", "하든지", "아니면", "이라면", "좋아", "알았어", "하는것도", "그만이다", "어쩔수", "없다", "하나", "일", "일반적으로", "일단", "한켠으로는", "오자마자", "이렇게되면", "이와같다면", "전부", "한마디", "한항목", "근거로", "하기에", "아울러", "하지", "않도록", "않기", "위해서", "이르기까지", "이", "되다", "로", "인하여", "까닭으로", "이유만으로", "이로", "인하여", "그래서", "이", "때문에", "그러므로", "그런", "까닭에", "알", "수", "있다", "결론을", "낼", "수", "있다", "으로", "인하여", "있다", "어떤것", "관계가", "있다", "관련이", "있다", "연관되다", "어떤것들", "에", "대해", "이리하여", "그리하여", "여부", "하기보다는", "하느니", "하면", "할수록", "운운", "이러이러하다", "하구나", "하도다", "다시말하면", "다음으로", "에", "있다", "에", "달려", "있다", "우리", "우리들", "오히려", "하기는한데", "어떻게", "어떻해", "어찌됏어", "어때", "어째서", "본대로", "자", "이", "이쪽", "여기", "이것", "이번", "이렇게말하자면", "이런", "이러한", "이와", "같은", "요만큼", "요만한", "것", "얼마", "안", "되는", "것", "이만큼", "이", "정도의", "이렇게", "많은", "것", "이와", "같다", "이때", "이렇구나", "것과", "같이", "끼익", "삐걱", "따위", "와", "같은", "사람들", "부류의", "사람들", "왜냐하면", "중의하나", "오직", "오로지", "에", "한하다", "하기만", "하면", "도착하다", "까지", "미치다", "도달하다", "정도에", "이르다", "할", "지경이다", "결과에", "이르다", "관해서는", "여러분", "하고", "있다", "한", "후", "혼자", "자기", "자기집", "자신", "우에", "종합한것과같이", "총적으로", "보면", "총적으로", "말하면", "총적으로", "대로", "하다", "으로서", "참", "그만이다", "할", "따름이다", "쿵", "탕탕", "쾅쾅", "둥둥", "봐", "봐라", "아이야", "아니", "와아", "응", "아이", "참나", "년", "월", "일", "령", "영", "일", "이", "삼", "사", "오", "육", "륙", "칠", "팔", "구", "이천육", "이천칠", "이천팔", "이천구", "하나", "둘", "셋", "넷", "다섯", "여섯", "일곱", "여덟", "아홉", "령", "영"]

def load_stop_words(stop_word_file):
    """
    Utility function to load stop words from a file and return as a list of words
    @param stop_word_file Path and file name of a file containing stop words.
    @return list A list of stop words.
    """
    stop_words = []
    for line in open(stop_word_file):
        if line.strip()[0:1] != "#":
            for word in line.split():  # in case more than one per line
                stop_words.append(word)
    return stop_words

def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False

def separate_words(text, min_word_return_size):
    """
    Utility function to return a list of all words that are have a length greater than a specified number of characters.
    @param text The text that must be split in to words.
    @param min_word_return_size The minimum no of characters a word must have to be included.
    """
    #mecab = Mecab()
    twitter = Twitter()
    #splitter = re.compile('[^a-zA-Z가-힣0-9_\\+\\-/]')
    words = []
    #for single_word in splitter.split(text):
    for single_word in twitter.nouns(text):
        current_word = single_word.strip().lower()
        #leave numbers in phrase, but don't count as words, since they tend to invalidate scores of their phrases
        if len(current_word) > min_word_return_size and current_word != '' and not is_number(current_word):
            words.append(current_word)
    return words

def split_sentences(text):
    """
    Utility function to return a list of sentences.
    @param text The text that must be split in to sentences.
    """
    sentence_delimiters = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')
    sentences = sentence_delimiters.split(text)
    return sentences

def build_stop_word_regex(stop_word_file_path):
#def build_stop_word_regex():
    stop_word_list = load_stop_words(stop_word_file_path)
#    stop_word_list = _stopwords
    stop_word_regex_list = []
    for word in stop_word_list:
        #word_regex = r'\b' + word + r'(?![\w-])'  # added look ahead for hyphen
        #stop_word_regex_list.append(word_regex)
        stop_word_regex_list.append(word)
    #stop_word_pattern = re.compile('|'.join(stop_word_regex_list), re.IGNORECASE)
    stop_word_pattern = re.compile('|'.join(stop_word_regex_list))
    return stop_word_pattern


def generate_candidate_keywords(sentence_list, stopword_pattern):
    phrase_list = []
    for s in sentence_list:
        tmp = re.sub(stopword_pattern, '|', s.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            if phrase != "":
                phrase_list.append(phrase)
    return phrase_list


def calculate_word_scores(phraseList):
    word_frequency = {}
    word_degree = {}
    mecab=Mecab()
    for phrase in phraseList:
        #word_list = separate_words(phrase, 0)
        word_list = mecab.nouns(phrase)
        word_list_length = len(word_list)
        word_list_degree = word_list_length - 1
        #if word_list_degree > 3: word_list_degree = 3 #exp.
        for word in word_list:
            word_frequency.setdefault(word, 0)
            word_frequency[word] += 1
            word_degree.setdefault(word, 0)
            word_degree[word] += word_list_degree  #orig.
            #word_degree[word] += 1/(word_list_length*1.0) #exp.
    for item in word_frequency:
        word_degree[item] = word_degree[item] + word_frequency[item]

    # Calculate Word scores = deg(w)/frew(w)
    word_score = {}
    for item in word_frequency:
        word_score.setdefault(item, 0)
        word_score[item] = word_degree[item] / (word_frequency[item] * 1.0)  #orig.
    #word_score[item] = word_frequency[item]/(word_degree[item] * 1.0) #exp.
    return word_score


def generate_candidate_keyword_scores(phrase_list, word_score):
    keyword_candidates = {}
    for phrase in phrase_list:
        keyword_candidates.setdefault(phrase, 0)
        word_list = separate_words(phrase, 0)
        candidate_score = 0
        for word in word_list:
            candidate_score += word_score[word]
        keyword_candidates[phrase] = candidate_score
    return keyword_candidates

In [172]:
rake_sentence = df_ten.apply(lambda row: split_sentences(row['article']), axis=1)
stoppath = "stopwordlist-kr.txt"  #SMART stoplist misses some of the lower-scoring keywords in Figure 1.5, which means that the top 1/3 cuts off one of the 4.0 score words in Table 1.1
stopwordpattern = build_stop_word_regex(stoppath)
for test in rake_sentence:
    phraseList = generate_candidate_keywords(test, stopwordpattern)
    wordscores = calculate_word_scores(phraseList)
    keywordcandidates = generate_candidate_keyword_scores(phraseList, wordscores)
    sortedKeywords = sorted(keywordcandidates, key=keywordcandidates.get, reverse=True)
    #totalKeywords = len(sortedKeywords)
    print(sortedKeywords)


KeyError: '전대'

In [135]:
splitter = re.compile('[^a-zA-Z가-힣0-9_\\+\\-/]')
splitter.split("양강구도로 가게 되면 국민은 우리나라 미래를 위해 다자구도하에서도 결선투표처럼 50% 이상의 지지를 받는 대통령을 당선시켜야 국정이 안정적으로 운영된다고 판단할")

['양강구도로',
 '가게',
 '되면',
 '국민은',
 '우리나라',
 '미래를',
 '위해',
 '다자구도하에서도',
 '결선투표처럼',
 '50',
 '',
 '이상의',
 '지지를',
 '받는',
 '대통령을',
 '당선시켜야',
 '국정이',
 '안정적으로',
 '운영된다고',
 '판단할']

In [156]:
word = '와|는'
stop_word_pattern = re.compile(word)
word_regex = r'\b' + word + r'(?![\w-])'
text = "대선 구도는 안철수와 문재인 두 사람 중 누구에 의한 정권교체가 나은 것인가다"
re.sub(stop_word_pattern, '|', text)

'대선 구도| 안철수| 문재인 두 사람 중 누구에 의한 정권교체가 나은 것인가다'